In [1]:
from seligator.main import train_and_get, Seligator
from seligator.common.load_save import load
from seligator.prediction.tests import run_tests

params = load("./seligator/siamese.json")
params["reader_kwargs"]["batch_size"] = 16
params["reader_kwargs"]["dev_batch_size"] = 32
#params["batches_per_epoch"] = 5


2021-08-17 17:23:44.752243: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
seligator, reader, train, dev = Seligator.init_from_params(**params)
train_kwargs = dict(patience=4, num_epochs=50, lr=1e-3, optimizer="AdamW")
print(seligator.model)

/home/thibault/dev/est-lascivuum-non-est/env/lib/python3.8/site-packages/allennlp/data/token_indexers/token_characters_indexer.py:55: UserWarning: You are using the default value (0) of `min_padding_length`, which can cause some subtle bugs (more info see https://github.com/allenai/allennlp/issues/1954). Strongly recommend to set a value, usually the maximum size of the convolutional layer size when using CnnEncoder.
  warnings.warn(


building vocab:   0%|          | 0/21955 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/155131 [00:00<?, ?it/s]

TripletClassifier(
  (_loss): TripletMarginLoss(
    (distance): LpDistance()
    (reducer): AvgNonZeroReducer()
  )
  (_miner): BatchHardMiner(
    (distance): LpDistance()
  )
  (_all_miner): BatchEasyHardMiner(
    (distance): LpDistance()
  )
  (left_encoder): MixedEmbeddingEncoder(
    (_emb_dropout): Dropout(p=0.3, inplace=False)
    (features_embedder): FeatureAndTextEmbedder(
      (text_embedder): BasicTextFieldEmbedder(
        (token_embedder_token): Embedding()
        (token_embedder_lemma): Embedding()
        (token_embedder_lemma_char): TokenCharactersEncoder(
          (_embedding): TimeDistributed(
            (_module): Embedding()
          )
          (_encoder): TimeDistributed(
            (_module): LstmSeq2VecEncoder(
              (_module): LSTM(100, 150, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
            )
          )
          (_dropout): Dropout(p=0.3, inplace=False)
        )
      )
      (feature_embedder): Linear(in_features=5

/home/thibault/dev/est-lascivuum-non-est/env/lib/python3.8/site-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [3]:
model = train_and_get(seligator.model, train, dev, **train_kwargs)
data, img = run_tests(
    "dataset/main/test.txt",
    dataset_reader=reader, model=model, dump=f"siamese.miner.best.csv"
)
#seligator.save_model("siamese_saved_model")

---> Epochs:   50
---> Patience: 4


  0%|          | 0/1373 [00:00<?, ?it/s]

  0%|          | 0/86 [00:00<?, ?it/s]

torch.Size([818]) torch.Size([818]) torch.Size([174])


IndexError: shape mismatch: indexing tensors could not be broadcast together with shapes [818], [174]